In [1]:
import os
import torch
import monai
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd, Orientationd, ScaleIntensityRanged, CropForegroundd,
    RandCropByPosNegLabeld, RandFlipd, RandRotate90d, RandShiftIntensityd, EnsureTyped, DivisiblePadd
)
from monai.data import DataLoader, CacheDataset
from monai.networks.nets import SwinUNETR
from monai.utils import set_determinism
from monai.data.image_reader import NibabelReader
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
 # Define directories
train_path = 'dataset/MICCAI_BraTS2020_TrainingData/'
# val_path = 'dataset/MICCAI_BraTS2020_ValidationData/'
modality_keys = ["flair"]
in_channels = len(modality_keys)
out_channels = len(modality_keys) 

In [3]:
def create_data_list(data_dir, patient_ids, lables, modality_keys):
    data_list = []
    for idx, patient in enumerate(patient_ids):
        patient_dir = f"{data_dir}{patient}/"
        if os.path.isdir(patient_dir):
            data_dict = {key: os.path.join(patient_dir, f"{patient}_{key}.nii") for key in modality_keys}
            data_dict['lable'] = lables[idx]
            data_list.append(data_dict)

    return data_list

In [4]:
def preprocess_labels(csv_file_path):
    df = pd.read_csv(csv_file_path)
    
    def extract_number(value):
        if isinstance(value, str):
            match = re.search(r'\d+', value)
            return int(match.group()) if match else None
        return value

    df['Survival_days'] = df['Survival_days'].apply(extract_number)
    df['Survival_days'] = pd.to_numeric(df['Survival_days'], errors='coerce')
    df = df.dropna(subset=['Survival_days'])
    df['Survival_days'] = df['Survival_days'].astype(int)
    
    return df['Survival_days'].values, df['Brats20ID'].values

In [5]:
all_labels, all_data_ids = preprocess_labels(f'{train_path}/survival_info.csv')
train_id, val_id, train_labels, val_labels = train_test_split(all_data_ids, all_labels, test_size=0.2, random_state=42)

train_data_list = create_data_list(train_path, train_id, train_labels, modality_keys)
val_data_list = create_data_list(train_path, val_id, val_labels, modality_keys)


# since we dont have lables for validation dataset not using it for validation
# val_data_list = create_data_list(val_path, modality_keys)
# val_labels = preprocess_labels(f'{val_path}/survival_evaluation.csv')



In [6]:
train_data_list[0]

{'flair': 'dataset/MICCAI_BraTS2020_TrainingData/BraTS20_Training_119/BraTS20_Training_119_flair.nii',
 'lable': 2}

In [7]:
def get_transforms(modality_keys, pixdim=(2.0, 2.0, 2.0), spatial_size=(64, 64, 64)):
    transforms = Compose(
        [
            LoadImaged(keys=modality_keys, reader=NibabelReader()),
            EnsureChannelFirstd(keys=modality_keys),
            Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
            Orientationd(keys=modality_keys, axcodes="RAS"),
            ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
            DivisiblePadd(keys=modality_keys, k=32),
            RandCropByPosNegLabeld(keys=modality_keys, label_key=modality_keys[0], spatial_size=spatial_size, pos=1, neg=1, num_samples=1, image_key=modality_keys[0], image_threshold=0),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=0),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=1),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=2),
            RandRotate90d(keys=modality_keys, prob=0.5, max_k=3),
            RandShiftIntensityd(keys=modality_keys, offsets=0.10, prob=0.5),
            EnsureTyped(keys=modality_keys),
        ]
    )
    return transforms

In [8]:
def get_val_transforms(modality_keys, pixdim=(2.0, 2.0, 2.0)):
    transforms = Compose(
        [
            LoadImaged(keys=modality_keys, reader=NibabelReader()),
            EnsureChannelFirstd(keys=modality_keys),
            Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
            Orientationd(keys=modality_keys, axcodes="RAS"),
            ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
            DivisiblePadd(keys=modality_keys, k=32),
            EnsureTyped(keys=modality_keys),
        ]
    )
    return transforms

In [9]:
train_transforms = get_transforms(modality_keys)
val_transforms = get_val_transforms(modality_keys)


In [10]:
train_ds = CacheDataset(
    data=train_data_list,
    transform=train_transforms,
    cache_rate=0.5,
    num_workers=4,
)
train_loader = DataLoader(train_ds,  batch_size=2, shuffle=True, num_workers=4)


Loading dataset: 100%|██████████| 94/94 [00:21<00:00,  4.30it/s]


In [11]:
# for data in train_loader:
#     print(data['flair'].shape)

In [12]:
val_ds = CacheDataset(
    data=val_data_list,
    transform=val_transforms,
    cache_rate=0.5,
    num_workers=4,
)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=2)

Loading dataset: 100%|██████████| 23/23 [00:06<00:00,  3.52it/s]


In [13]:
# Load the pretrained model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SwinUNETR(
    img_size=(64, 64, 64),
    in_channels=in_channels,
    out_channels=out_channels,
    feature_size=48,
    use_checkpoint=True,
).to(device)


# Load the saved weights
modality_used = "_".join(modality_keys)
model_save_path = f"model_saved/swin_unetr_{modality_used}_temp.pth"

model.load_state_dict(torch.load(model_save_path))
model.eval()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


SwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(1, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=48, out_features=144, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=48, out_features=48, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=48, out_features=192, bias=True)
              (linear2): Linear(in_feature

In [14]:
# # 2. Modify the SwinUNETR class to add a method for feature extraction
# class FeatureExtractorSwinUNETR(SwinUNETR):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
    
#     def extract_features(self, x):
#         print(" at start",x.size())
#         hidden_states = self.swinViT(x, self.normalize)
#         enc0 = self.encoder1(x)
#         print(" after enco 1",x.size())
#         enc1 = self.encoder2(hidden_states[0])
#         print(" after enco 2",enc1.size())
#         enc2 = self.encoder3(hidden_states[1])
#         print(" after enco 3",enc2.size())
#         enc3 = self.encoder4(hidden_states[2])
#         print(" after enco 4",enc3.size())
#         # dec3 = self.encoder10(hidden_states[3])
#         # print(" after dec3",dec3.size())
#         # return torch.cat([enc0, enc1, enc2, enc3, dec3], dim=1)
#         return torch.cat([enc0, enc1, enc2, enc3], dim=1)

In [15]:
class FeatureExtractorSwinUNETR(SwinUNETR):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def extract_features(self, x):
        hidden_states = self.swinViT(x, self.normalize)
        
        # Adaptive pooling to resize all hidden states to a common size
        pooled_states = []
        for state in hidden_states:
            # Adaptive average pooling to 4x4x4
            pooled = F.adaptive_avg_pool3d(state, (4, 4, 4))
            pooled_states.append(pooled)
        
        # Concatenate the pooled states
        return torch.cat(pooled_states, dim=1)


In [16]:
#  Create an instance of the feature extractor
feature_extractor = FeatureExtractorSwinUNETR(
    img_size=(64, 64, 64),
    in_channels=in_channels,
    out_channels=out_channels,
    feature_size=48,
    use_checkpoint=True,
).to(device)
feature_extractor.load_state_dict(model.state_dict())
feature_extractor.eval()

FeatureExtractorSwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(1, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=48, out_features=144, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=48, out_features=48, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=48, out_features=192, bias=True)
              (linear2): L

In [17]:

# # 3. Extract features from your dataset
# def extract_features_from_dataset(dataset, feature_extractor):
#     features = []
#     labels = []
    
#     for data in dataset:
#         inputs = torch.cat([data[key] for key in modality_keys], dim=1).to(device)
#         with torch.no_grad():
#             feature = feature_extractor.extract_features(inputs)
        
#         # Assuming the feature is 5D (batch, channels, depth, height, width)
#         # We'll flatten it to 2D (batch, features)
#         feature = feature.view(feature.size(0), -1)
        
#         features.append(feature.cpu().numpy())
#         labels.append(data['label'].cpu().numpy())  # Assuming you have labels in your dataset
    
#     return np.concatenate(features), np.concatenate(labels)


In [18]:
def extract_features_from_dataset(dataloader, feature_extractor):
    features = []
    labels = []
    
    for batch in dataloader:
        inputs = torch.cat([batch[key] for key in modality_keys], dim=1).to(device)
        with torch.no_grad():
            feature = feature_extractor.extract_features(inputs)
        
        # Global average pooling to reduce spatial dimensions
        feature = torch.mean(feature, dim=[2, 3, 4])
        
        features.append(feature.cpu().numpy())
        # labels.append(batch['label'].cpu().numpy())  # Assuming you have labels in your dataset
    
    return np.concatenate(features)


In [19]:

# Extract features for training and validation sets
train_features = extract_features_from_dataset(train_loader, feature_extractor)
val_features = extract_features_from_dataset(val_loader, feature_extractor)


/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(


In [20]:
# Define XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)


In [21]:
# Define hyperparameter search space
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [1, 2, 3, 4, 5]
}


In [22]:

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=25, cv=5, random_state=42, n_jobs=-1)


In [23]:

# Fit the model
random_search.fit(train_features, train_labels)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='mlogloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learnin...
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=42, ...),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                             1.0],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 4, 5, 6, 7, 8],
                                        'min_child_weight': [1, 2, 3, 4, 5],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]},
                   random_state=42)

In [24]:
# Get the best model
best_xgb_model = random_search.best_estimator_
print("Best parameters found: ", random_search.best_params_)
 

Best parameters found:  {'subsample': 0.7, 'n_estimators': 400, 'min_child_weight': 4, 'max_depth': 6, 'learning_rate': 0.01, 'colsample_bytree': 0.6}


In [25]:
# Make predictions on validation set
val_pred = best_xgb_model.predict(val_features)


In [26]:
# Evaluate the model
accuracy = accuracy_score(val_labels, val_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(val_labels, val_pred))

Accuracy: 0.2765957446808511


/home/m1/23CS60R48/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        15
           2       0.28      1.00      0.43        13

    accuracy                           0.28        47
   macro avg       0.09      0.33      0.14        47
weighted avg       0.08      0.28      0.12        47



/home/m1/23CS60R48/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/m1/23CS60R48/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# Feature importance
feature_importance = best_xgb_model.feature_importances_
sorted_idx = np.argsort(feature_importance)
sorted_features = [f"Feature {i}" for i in sorted_idx]
sorted_importance = feature_importance[sorted_idx]

In [28]:
train_features.shape


(188, 1488)

In [29]:
val_features.shape

(47, 1488)